# Моделирование охлаждения NVT ансамбля <a class="tocSkip">

   - Система: $N = 1372$ частиц;
   - Ячейка моделирования: $V = 12.25\sigma\times12.25\sigma\times12.25\sigma$;
   - Потенциал взаимодействия: Леннард-Джонс;
   - Ансамбль: $NVT$;
   - Начальная температура: $T_i = 1.3~\varepsilon / k_B$;
   - Конечная температура: $T_f = 10^{-4}~\varepsilon / k_B$;
   - Скорости охлаждения: $\gamma = 2\times10^{-5}~\varepsilon / k_B\tau$;
   - Термостат: масштабирование скоростей;
   - Толщина сферического слоя (список Верле): $\Delta r_s = 0.3\sigma $;
   - Временной шаг: $\Delta t = 0.005\tau$;

## Imports

In [ ]:
from datetime import datetime
from pathlib import Path
import sys

BASE_DIR = Path('.').resolve().parent
sys.path.append(str(BASE_DIR))

In [ ]:
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd
from scipy import interpolate
from optimizers_af.optimizers import HookeJeeves, RadialMovementOptimization

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
%load_ext autoreload
%autoreload 2

# from rmo import RadialMovementOptimization
from scripts_old.plotter import Plotter, get_temperature_legend
import scripts_old.postprocessing as pp
from scripts_old.properties.ovito_procedures import OvitoProcessor

## Constants

In [ ]:
START_TIME = datetime.now()
CURRENT_DATA = '2022-09-25_velocity_scaling_HV_2e-5_T_01e-4'
PATH_TO_CURRENT_DATA = BASE_DIR / 'data' / CURRENT_DATA
PATH_TO_CURRENT_PLOTS = BASE_DIR / 'plots' / 'article'
PLOT_FILENAME_POSTFIX = 'HV_2e-5'
PATH_TO_CURRENT_DATA

In [ ]:
CELL_DIMENSIONS = np.ones(3) * 12.25

## Functions

In [ ]:
@njit
def math_round(value):
    rest = value - int(value)
    if rest >= 0.5 and value >= 0:
        return float(int(value) + 1)
    if rest <= -0.5 and value < 0:
        return float(int(value) - 1)
    return float(int(value))

@njit
def get_boundary_conditions(
        particles_number: int,
        positions: np.ndarray,
):
    for i in range(particles_number):
        for j in range(3):
            if positions[i][j] >= CELL_DIMENSIONS[j] / 2.0:
                positions[i][j] -= (
                        math_round(positions[i][j] / CELL_DIMENSIONS[j])
                        * CELL_DIMENSIONS[j]
                )
            if positions[i][j] < -CELL_DIMENSIONS[j] / 2.0:
                positions[i][j] -= (
                        math_round(positions[i][j] / CELL_DIMENSIONS[j])
                        * CELL_DIMENSIONS[j]
                )
    return positions


def calculate_rdf(positions):
    radii, rdf = OvitoProcessor(
        positions=positions, 
        cell_dimensions=CELL_DIMENSIONS,
    ).get_rdf()
    return radii, rdf


def generate_random_state() -> None:
    particles_number = 1372
    cell_dimensions = 12.25 * np.ones(3)
    return (np.random.random((particles_number, 3)) - 0.5) * cell_dimensions

## Data reading

In [ ]:
system_configuration_path = ''
for filename in PATH_TO_CURRENT_DATA.iterdir():
    if filename.match('system_configuration*.csv'):
        system_configuration_path = str(PATH_TO_CURRENT_DATA / filename)
        break

initial_configuration = pd.read_csv(system_configuration_path, sep=';')[['x', 'y', 'z']].to_numpy()

# initial_configuration = generate_random_state()
initial_rdf = calculate_rdf(initial_configuration)[1]
print(initial_configuration.shape, initial_rdf.shape)

In [ ]:
predicted_rdf_df = pd.read_csv(str(PATH_TO_CURRENT_DATA / 'predicted_rdf.csv'), sep=';')
all_radiuses = predicted_rdf_df['radius'].values
predicted_rdf = predicted_rdf_df['predicted_rdf'].values
observed_rdf = predicted_rdf_df['observed_rdf'].values
nz_indices = np.where(predicted_rdf.any())[0]

## Configuration optimization

In [ ]:
def optimized_func(positions):
    radii, exp_rdf = calculate_rdf(
        positions=positions.reshape(1372, 3),
    )
    result = mean_squared_error(predicted_rdf, exp_rdf)
    collided_number = exp_rdf[:nz_indices[0]].sum()
    return result + collided_number

In [ ]:
RMO = RadialMovementOptimization(
    func=optimized_func,
    generations_number=20,
    particles_number=500,
    bounds=[(-6.125, 6.125) for _ in range(1372 * 3)],
    c_parameters=(0.9, 1.0),
    weight_limits=(0, 1),
    scale=10,
)
new_positions = RMO.run(
    point=initial_configuration.flatten()
)[0]

In [ ]:
HJ = HookeJeeves(
    func=optimized_func,
    accuracy=1e-2,
    initial_step_size=1,
    step_reduction_factor=0.5,
)

In [ ]:
HJ.run(point=new_positions)

In [ ]:
new_positions_hj = HJ.history[-1][0].reshape(1372, 3)

In [ ]:
hj_radiuses, hj_rdf = calculate_rdf(new_positions_hj)

plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=0,
        right=6,
        bottom=-0.1,
        top=3,
    ),
    labels=('radius', 'rdf'),
)

s = 20
plotter.ax.plot(all_radiuses, observed_rdf, label='Observed')
plotter.ax.plot(all_radiuses, predicted_rdf, label='Predicted')
plotter.ax.plot(hj_radiuses, hj_rdf, label='HJ')
plotter.set_major_locators(1, 0.5)
plotter.set_minor_locators(0.2, 0.1)
plotter.get_legend()
plt.show()

## End

In [ ]:
print(f'Execution Time: {datetime.now() - START_TIME}')